# OBRIGATORIA


In [2]:
%pip install pandas
%pip install matplot
%pip install mlxtend

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules, apriori
from importlib import reload
import numpy as np
import csv

plt=reload(plt)

# FILE


In [4]:
FILE_NAME_REFACTORING = 'refactorings-auto.csv'
FILE_NAME_COMMENTS = 'comments-auto.csv'

# TRABALHANDO NOS CALCULOS

In [6]:
DF_COMMENTS = pd.read_csv(FILE_NAME_COMMENTS)

# CONTAR QUANTOS DIFERENTE TEM EM 19_CLASSPATH
DF_C = DF_COMMENTS['19_CLASSPATH'].value_counts()

DF_COMMENTS.drop(columns=['15_TYPE', '16_STARTLINE', '17_ENDLINE', '19_CLASSPATH', '11_HASH', '12_PARENTSHASH'], inplace=True)
DF_COMMENTS

,13_HASH_CLASSPATH,14_PARENTHASH_CLASSPATH,18_SEGMENTOS,20_ISBLOCKCOMMENT,21_ISLINECOMMENT,22_ISJAVADOCCOMMENT,23_ISORPHANCOMMENTA
0,925fbaacbe8e74dd550ee3ec81a45b81e6ba09d3/commo...,fbedbb5c385fa2a898ab7e14100fdca242c841ba/commo...,5,0,0,1,0
1,925fbaacbe8e74dd550ee3ec81a45b81e6ba09d3/commo...,fbedbb5c385fa2a898ab7e14100fdca242c841ba/commo...,5,0,0,1,0
2,925fbaacbe8e74dd550ee3ec81a45b81e6ba09d3/commo...,fbedbb5c385fa2a898ab7e14100fdca242c841ba/commo...,11,0,0,1,0
3,925fbaacbe8e74dd550ee3ec81a45b81e6ba09d3/commo...,fbedbb5c385fa2a898ab7e14100fdca242c841ba/commo...,1,0,1,0,0
4,925fbaacbe8e74dd550ee3ec81a45b81e6ba09d3/commo...,fbedbb5c385fa2a898ab7e14100fdca242c841ba/commo...,1,0,1,0,0
...,...,...,...,...,...,...,...
2881979,60c5fb24011afb842faccde4ef362df3ba023fb4/value...,d528f7f3d2243c73c97189d35d90638b17722823/value...,1,0,1,0,0
2881980,60c5fb24011afb842faccde4ef362df3ba023fb4/value...,d528f7f3d2243c73c97189d35d90638b17722823/value...,1,0,1,0,0
2881981,60c5fb24011afb842faccde4ef362df3ba023fb4/value...,d528f7f3d2243c73c97189d35d90638b17722823/value...,6,0,0,1,0
2881982,60c5fb24011afb842faccde4ef362df3ba023fb4/value...,d528f7f3d2243c73c97189d35d90638b17722823/value...,1,0,1,0,0


In [7]:
DF_C

19_CLASSPATH
common/src/main/java/com/google/auto/common/MoreTypes.java                                  125924
value/src/main/java/com/google/auto/value/processor/AutoValueProcessor.java                 107285
value/src/main/java/com/google/auto/value/processor/AutoValueishProcessor.java               93974
common/src/main/java/com/google/auto/common/Overrides.java                                   92780
value/src/main/java/com/google/auto/value/processor/BuilderMethodClassifier.java             91009
                                                                                             ...  
generator/src/test/java/com/google/autofactory/testing/SimpleClassPassedDeps.java                1
value/src/test/java/com/google/auto/value/SimpleValueTypeTest.java                               1
generator/src/test/java/com/google/autofactory/testing/SimpleClassProvidedDeps.java              1
factory/src/test/resources/expected/CustomNullableFactory.java                                  

In [32]:

# Supondo que DF_COMMENTS seja o seu DataFrame
# Excluímos a coluna '14_PARENTHASH_CLASSPATH' do DataFrame temporariamente
df_temp = DF_COMMENTS.drop(columns=["14_PARENTHASH_CLASSPATH"])

# Agrupamos pelo '13_HASH_CLASSPATH' e somamos as outras colunas
agrupado = df_temp.groupby(["13_HASH_CLASSPATH"]).sum().reset_index()

# Adiciona a coluna '14_PARENTHASH_CLASSPATH' de volta
agrupado = agrupado.merge(DF_COMMENTS[['13_HASH_CLASSPATH', '14_PARENTHASH_CLASSPATH']].drop_duplicates(), on='13_HASH_CLASSPATH', how='left')

# Ordena pelo '13_HASH_CLASSPATH' para manter a ordem original
DF_COMMENTS = DF_COMMENTS[['13_HASH_CLASSPATH', '14_PARENTHASH_CLASSPATH']].drop_duplicates().sort_values(by='13_HASH_CLASSPATH')

# Reordenar o DataFrame resultante com base na ordem original
agrupado = agrupado.set_index('13_HASH_CLASSPATH').loc[DF_COMMENTS['13_HASH_CLASSPATH']].reset_index()

# Renomeando as colunas para os novos nomes
agrupado.rename(columns={
    "18_SEGMENTOS": "QNT_18_SEGMENTOS",
    "20_ISBLOCKCOMMENT": "QNT_20_ISBLOCKCOMMENT",
    "21_ISLINECOMMENT": "QTN_21_ISLINECOMMENT",
    "22_ISJAVADOCCOMMENT": "QTN_22_ISJAVADOCCOMMENT",
    "23_ISORPHANCOMMENTA": "QTN_23_ISORPHANCOMMENTA"

}, inplace=True)


agrupado.to_csv('agrupado.csv', index=False)
agrupado

,13_HASH_CLASSPATH,QNT_18_SEGMENTOS,QNT_20_ISBLOCKCOMMENT,QTN_21_ISLINECOMMENT,QTN_22_ISJAVADOCCOMMENT,QTN_23_ISORPHANCOMMENTA,14_PARENTHASH_CLASSPATH
0,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,57,0,2,9,0,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...
1,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,27,0,5,4,2,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...
2,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,161,0,10,28,2,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...
3,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,164,0,7,21,3,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...
4,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,30,0,0,5,0,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...
...,...,...,...,...,...,...,...
273047,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,12,1,2,1,3,bdaa2302030669589545dc624c8c16a1d7f939db/test-...
273048,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,6,0,0,1,0,bdaa2302030669589545dc624c8c16a1d7f939db/test-...
273049,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,6,0,0,1,0,bdaa2302030669589545dc624c8c16a1d7f939db/test-...
273050,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,6,0,1,1,1,bdaa2302030669589545dc624c8c16a1d7f939db/test-...


In [33]:
# Juntando o DataFrame consigo mesmo para calcular as diferenças
merged_df = pd.merge(agrupado, agrupado, left_on="14_PARENTHASH_CLASSPATH", right_on="13_HASH_CLASSPATH", suffixes=('', '_PARENTE'), how='left')


merged_df.to_csv('merged_df.csv', index=False)

merged_df.drop(columns=['14_PARENTHASH_CLASSPATH_PARENTE'], inplace=True)

merged_df

,13_HASH_CLASSPATH,QNT_18_SEGMENTOS,QNT_20_ISBLOCKCOMMENT,QTN_21_ISLINECOMMENT,QTN_22_ISJAVADOCCOMMENT,QTN_23_ISORPHANCOMMENTA,14_PARENTHASH_CLASSPATH,13_HASH_CLASSPATH_PARENTE,QNT_18_SEGMENTOS_PARENTE,QNT_20_ISBLOCKCOMMENT_PARENTE,QTN_21_ISLINECOMMENT_PARENTE,QTN_22_ISJAVADOCCOMMENT_PARENTE,QTN_23_ISORPHANCOMMENTA_PARENTE
0,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,57,0,2,9,0,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,57.0,0.0,2.0,9.0,0.0
1,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,27,0,5,4,2,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,27.0,0.0,5.0,4.0,2.0
2,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,161,0,10,28,2,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,161.0,0.0,10.0,28.0,2.0
3,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,164,0,7,21,3,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,164.0,0.0,7.0,21.0,3.0
4,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,30,0,0,5,0,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,30.0,0.0,0.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
273047,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,12,1,2,1,3,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,12.0,1.0,2.0,1.0,2.0
273048,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,6,0,0,1,0,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,9.0,2.0,1.0,1.0,0.0
273049,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,6,0,0,1,0,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,6.0,0.0,0.0,1.0,0.0
273050,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,6,0,1,1,1,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,5.0,0.0,0.0,1.0,0.0


# CALCULO DIFERENCAS

In [34]:

# Calculando as diferenças
merged_df["DIFERENCA_QNT_20_ISBLOCKCOMMENT"] = merged_df["QNT_20_ISBLOCKCOMMENT"] - merged_df["QNT_20_ISBLOCKCOMMENT_PARENTE"]
merged_df["DIFERENCA_QTN_21_ISLINECOMMENT"] = merged_df["QTN_21_ISLINECOMMENT"] - merged_df["QTN_21_ISLINECOMMENT_PARENTE"]
merged_df["DIFERENCA_QTN_22_ISJAVADOCCOMMENT"] = merged_df["QTN_22_ISJAVADOCCOMMENT"] - merged_df["QTN_22_ISJAVADOCCOMMENT_PARENTE"]
merged_df["DIFERENCA_QTN_23_ISORPHANCOMMENTA"] = merged_df["QTN_23_ISORPHANCOMMENTA"] - merged_df["QTN_23_ISORPHANCOMMENTA_PARENTE"]
merged_df["DIFERENCA_QNT_18_SEGMENTOS"] =  merged_df["QNT_18_SEGMENTOS"] - merged_df["QNT_18_SEGMENTOS_PARENTE"]

merged_df

,13_HASH_CLASSPATH,QNT_18_SEGMENTOS,QNT_20_ISBLOCKCOMMENT,QTN_21_ISLINECOMMENT,QTN_22_ISJAVADOCCOMMENT,QTN_23_ISORPHANCOMMENTA,14_PARENTHASH_CLASSPATH,13_HASH_CLASSPATH_PARENTE,QNT_18_SEGMENTOS_PARENTE,QNT_20_ISBLOCKCOMMENT_PARENTE,QTN_21_ISLINECOMMENT_PARENTE,QTN_22_ISJAVADOCCOMMENT_PARENTE,QTN_23_ISORPHANCOMMENTA_PARENTE,DIFERENCA_QNT_20_ISBLOCKCOMMENT,DIFERENCA_QTN_21_ISLINECOMMENT,DIFERENCA_QTN_22_ISJAVADOCCOMMENT,DIFERENCA_QTN_23_ISORPHANCOMMENTA,DIFERENCA_QNT_18_SEGMENTOS
0,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,57,0,2,9,0,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,57.0,0.0,2.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0
1,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,27,0,5,4,2,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,27.0,0.0,5.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0
2,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,161,0,10,28,2,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,161.0,0.0,10.0,28.0,2.0,0.0,0.0,0.0,0.0,0.0
3,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,164,0,7,21,3,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,164.0,0.0,7.0,21.0,3.0,0.0,0.0,0.0,0.0,0.0
4,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,30,0,0,5,0,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,30.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273047,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,12,1,2,1,3,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,12.0,1.0,2.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0
273048,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,6,0,0,1,0,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,9.0,2.0,1.0,1.0,0.0,-2.0,-1.0,0.0,0.0,-3.0
273049,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,6,0,0,1,0,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,6.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
273050,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,6,0,1,1,1,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,5.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0


In [35]:
# Selecionando as colunas desejadas
novo_df_com_diferencas = merged_df[[
    "13_HASH_CLASSPATH", "14_PARENTHASH_CLASSPATH",
    "QNT_20_ISBLOCKCOMMENT", "QTN_21_ISLINECOMMENT", "QTN_22_ISJAVADOCCOMMENT", "QTN_23_ISORPHANCOMMENTA",
    "DIFERENCA_QNT_20_ISBLOCKCOMMENT", "DIFERENCA_QTN_21_ISLINECOMMENT","DIFERENCA_QNT_18_SEGMENTOS",
    "DIFERENCA_QTN_22_ISJAVADOCCOMMENT", "DIFERENCA_QTN_23_ISORPHANCOMMENTA"
]]

# Removendo as linhas onde não há correspondência para 14_PARENTHASH_CLASSPATH
novo_df_com_diferencas.dropna(inplace=True)

novo_df_com_diferencas.to_csv('novo_df_com_diferencas.csv', index=False)

novo_df_com_diferencas

C:\Users\kleit\AppData\Local\Temp\ipykernel_11544\803076579.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  novo_df_com_diferencas.dropna(inplace=True)


,13_HASH_CLASSPATH,14_PARENTHASH_CLASSPATH,QNT_20_ISBLOCKCOMMENT,QTN_21_ISLINECOMMENT,QTN_22_ISJAVADOCCOMMENT,QTN_23_ISORPHANCOMMENTA,DIFERENCA_QNT_20_ISBLOCKCOMMENT,DIFERENCA_QTN_21_ISLINECOMMENT,DIFERENCA_QNT_18_SEGMENTOS,DIFERENCA_QTN_22_ISJAVADOCCOMMENT,DIFERENCA_QTN_23_ISORPHANCOMMENTA
0,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,0,2,9,0,0.0,0.0,0.0,0.0,0.0
1,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,0,5,4,2,0.0,0.0,0.0,0.0,0.0
2,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,0,10,28,2,0.0,0.0,0.0,0.0,0.0
3,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,0,7,21,3,0.0,0.0,0.0,0.0,0.0
4,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,0,0,5,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
273047,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,1,2,1,3,0.0,0.0,0.0,0.0,1.0
273048,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,0,0,1,0,-2.0,-1.0,-3.0,0.0,0.0
273049,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,0,0,1,0,0.0,0.0,0.0,0.0,0.0
273050,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,0,1,1,1,0.0,1.0,1.0,0.0,1.0


In [36]:
DF_REFACTORING = pd.read_csv(FILE_NAME_REFACTORING)
df = novo_df_com_diferencas
df.to_csv('df_refactorings.csv', index=False)
df

,13_HASH_CLASSPATH,14_PARENTHASH_CLASSPATH,QNT_20_ISBLOCKCOMMENT,QTN_21_ISLINECOMMENT,QTN_22_ISJAVADOCCOMMENT,QTN_23_ISORPHANCOMMENTA,DIFERENCA_QNT_20_ISBLOCKCOMMENT,DIFERENCA_QTN_21_ISLINECOMMENT,DIFERENCA_QNT_18_SEGMENTOS,DIFERENCA_QTN_22_ISJAVADOCCOMMENT,DIFERENCA_QTN_23_ISORPHANCOMMENTA
0,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,0,2,9,0,0.0,0.0,0.0,0.0,0.0
1,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,0,5,4,2,0.0,0.0,0.0,0.0,0.0
2,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,0,10,28,2,0.0,0.0,0.0,0.0,0.0
3,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,0,7,21,3,0.0,0.0,0.0,0.0,0.0
4,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,07f37b2be9e060f57eed3e3f5cdccab6a53f0927/commo...,0,0,5,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
273047,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,1,2,1,3,0.0,0.0,0.0,0.0,1.0
273048,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,0,0,1,0,-2.0,-1.0,-3.0,0.0,0.0
273049,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,0,0,1,0,0.0,0.0,0.0,0.0,0.0
273050,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,bdaa2302030669589545dc624c8c16a1d7f939db/test-...,0,1,1,1,0.0,1.0,1.0,0.0,1.0


In [37]:
df.drop(columns=['14_PARENTHASH_CLASSPATH', 'QNT_20_ISBLOCKCOMMENT',  'QTN_21_ISLINECOMMENT', 'QTN_22_ISJAVADOCCOMMENT', 'QTN_23_ISORPHANCOMMENTA'], inplace=True)
df

C:\Users\kleit\AppData\Local\Temp\ipykernel_11544\1637674147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['14_PARENTHASH_CLASSPATH', 'QNT_20_ISBLOCKCOMMENT',  'QTN_21_ISLINECOMMENT', 'QTN_22_ISJAVADOCCOMMENT', 'QTN_23_ISORPHANCOMMENTA'], inplace=True)


,13_HASH_CLASSPATH,DIFERENCA_QNT_20_ISBLOCKCOMMENT,DIFERENCA_QTN_21_ISLINECOMMENT,DIFERENCA_QNT_18_SEGMENTOS,DIFERENCA_QTN_22_ISJAVADOCCOMMENT,DIFERENCA_QTN_23_ISORPHANCOMMENTA
0,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0.0,0.0,0.0,0.0,0.0
1,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0.0,0.0,0.0,0.0,0.0
2,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0.0,0.0,0.0,0.0,0.0
3,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0.0,0.0,0.0,0.0,0.0
4,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
273047,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,0.0,0.0,0.0,0.0,1.0
273048,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,-2.0,-1.0,-3.0,0.0,0.0
273049,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,0.0,0.0,0.0,0.0,0.0
273050,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,0.0,1.0,1.0,0.0,1.0


In [38]:
# Criar uma nova coluna 'COMMENTS_CHANGE' com valores padrão como None
df['DIFERENCA_QNT_20_ISBLOCKCOMMENT_CHANGE'] = None
df['DIFERENCA_QNT_20_ISBLOCKCOMMENT_Aumentou'] = None
df['DIFERENCA_QNT_20_ISBLOCKCOMMENT_Diminuiu'] = None
df['DIFERENCA_QNT_20_ISBLOCKCOMMENT_Manteve'] = None

# Atualizar a coluna 'DIFERENCA_QNT_20_ISBLOCKCOMMENT_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QNT_20_ISBLOCKCOMMENT'] > 0, 'DIFERENCA_QNT_20_ISBLOCKCOMMENT_Aumentou'] = 1
df.loc[df['DIFERENCA_QNT_20_ISBLOCKCOMMENT'] < 0, 'DIFERENCA_QNT_20_ISBLOCKCOMMENT_Diminuiu'] = 1
df.loc[df['DIFERENCA_QNT_20_ISBLOCKCOMMENT'] == 0, 'DIFERENCA_QNT_20_ISBLOCKCOMMENT_Manteve'] = 1

# Atualizar a coluna 'DIFERENCA_QNT_20_ISBLOCKCOMMENT_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QNT_20_ISBLOCKCOMMENT'] > 0, 'DIFERENCA_QNT_20_ISBLOCKCOMMENT_CHANGE'] = df['DIFERENCA_QNT_20_ISBLOCKCOMMENT']
df.loc[df['DIFERENCA_QNT_20_ISBLOCKCOMMENT'] < 0, 'DIFERENCA_QNT_20_ISBLOCKCOMMENT_CHANGE'] = df['DIFERENCA_QNT_20_ISBLOCKCOMMENT']
df.loc[df['DIFERENCA_QNT_20_ISBLOCKCOMMENT'] == 0, 'DIFERENCA_QNT_20_ISBLOCKCOMMENT_CHANGE'] = 0

df['DIFERENCA_QNT_20_ISBLOCKCOMMENT_CHANGE'].fillna(0, inplace=True)
df['DIFERENCA_QNT_20_ISBLOCKCOMMENT_Aumentou'].fillna(0, inplace=True)
df['DIFERENCA_QNT_20_ISBLOCKCOMMENT_Diminuiu'].fillna(0, inplace=True)
df['DIFERENCA_QNT_20_ISBLOCKCOMMENT_Manteve'].fillna(0, inplace=True)
df.drop(columns=['DIFERENCA_QNT_20_ISBLOCKCOMMENT' , 'DIFERENCA_QNT_20_ISBLOCKCOMMENT_CHANGE'], inplace=True)
df

C:\Users\kleit\AppData\Local\Temp\ipykernel_11544\1950198288.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DIFERENCA_QNT_20_ISBLOCKCOMMENT_CHANGE'] = None
C:\Users\kleit\AppData\Local\Temp\ipykernel_11544\1950198288.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DIFERENCA_QNT_20_ISBLOCKCOMMENT_Aumentou'] = None
C:\Users\kleit\AppData\Local\Temp\ipykernel_11544\1950198288.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

,13_HASH_CLASSPATH,DIFERENCA_QTN_21_ISLINECOMMENT,DIFERENCA_QNT_18_SEGMENTOS,DIFERENCA_QTN_22_ISJAVADOCCOMMENT,DIFERENCA_QTN_23_ISORPHANCOMMENTA,DIFERENCA_QNT_20_ISBLOCKCOMMENT_Aumentou,DIFERENCA_QNT_20_ISBLOCKCOMMENT_Diminuiu,DIFERENCA_QNT_20_ISBLOCKCOMMENT_Manteve
0,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0.0,0.0,0.0,0.0,0,0,1
1,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0.0,0.0,0.0,0.0,0,0,1
2,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0.0,0.0,0.0,0.0,0,0,1
3,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0.0,0.0,0.0,0.0,0,0,1
4,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0.0,0.0,0.0,0.0,0,0,1
...,...,...,...,...,...,...,...,...
273047,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,0.0,0.0,0.0,1.0,0,0,1
273048,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,-1.0,-3.0,0.0,0.0,0,1,0
273049,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,0.0,0.0,0.0,0.0,0,0,1
273050,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,1.0,1.0,0.0,1.0,0,0,1


In [39]:
# Criar uma nova coluna 'COMMENTS_CHANGE' com valores padrão como None
df['DIFERENCA_QTN_21_ISLINECOMMENT_CHANGE'] = None
df['DIFERENCA_QTN_21_ISLINECOMMENT_Aumentou'] = None
df['DIFERENCA_QTN_21_ISLINECOMMENT_Diminuiu'] = None
df['DIFERENCA_QTN_21_ISLINECOMMENT_Manteve'] = None

# Atualizar a coluna 'DIFERENCA_QTN_21_ISLINECOMMENT_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QTN_21_ISLINECOMMENT'] > 0, 'DIFERENCA_QTN_21_ISLINECOMMENT_Aumentou'] = 1
df.loc[df['DIFERENCA_QTN_21_ISLINECOMMENT'] < 0, 'DIFERENCA_QTN_21_ISLINECOMMENT_Diminuiu'] = 1
df.loc[df['DIFERENCA_QTN_21_ISLINECOMMENT'] == 0, 'DIFERENCA_QTN_21_ISLINECOMMENT_Manteve'] = 1

# Atualizar a coluna 'DIFERENCA_QTN_21_ISLINECOMMENT_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QTN_21_ISLINECOMMENT'] > 0, 'DIFERENCA_QTN_21_ISLINECOMMENT_CHANGE'] = df['DIFERENCA_QTN_21_ISLINECOMMENT']
df.loc[df['DIFERENCA_QTN_21_ISLINECOMMENT'] < 0, 'DIFERENCA_QTN_21_ISLINECOMMENT_CHANGE'] = df['DIFERENCA_QTN_21_ISLINECOMMENT']
df.loc[df['DIFERENCA_QTN_21_ISLINECOMMENT'] == 0, 'DIFERENCA_QTN_21_ISLINECOMMENT_CHANGE'] = 0

df['DIFERENCA_QTN_21_ISLINECOMMENT_CHANGE'].fillna(0, inplace=True)
df['DIFERENCA_QTN_21_ISLINECOMMENT_Aumentou'].fillna(0, inplace=True)
df['DIFERENCA_QTN_21_ISLINECOMMENT_Diminuiu'].fillna(0, inplace=True)
df['DIFERENCA_QTN_21_ISLINECOMMENT_Manteve'].fillna(0, inplace=True)
df.drop(columns=['DIFERENCA_QTN_21_ISLINECOMMENT', 'DIFERENCA_QTN_21_ISLINECOMMENT_CHANGE'], inplace=True)
df

C:\Users\kleit\AppData\Local\Temp\ipykernel_11544\1011571628.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DIFERENCA_QTN_21_ISLINECOMMENT_CHANGE'] = None
C:\Users\kleit\AppData\Local\Temp\ipykernel_11544\1011571628.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DIFERENCA_QTN_21_ISLINECOMMENT_Aumentou'] = None
C:\Users\kleit\AppData\Local\Temp\ipykernel_11544\1011571628.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

,13_HASH_CLASSPATH,DIFERENCA_QNT_18_SEGMENTOS,DIFERENCA_QTN_22_ISJAVADOCCOMMENT,DIFERENCA_QTN_23_ISORPHANCOMMENTA,DIFERENCA_QNT_20_ISBLOCKCOMMENT_Aumentou,DIFERENCA_QNT_20_ISBLOCKCOMMENT_Diminuiu,DIFERENCA_QNT_20_ISBLOCKCOMMENT_Manteve,DIFERENCA_QTN_21_ISLINECOMMENT_Aumentou,DIFERENCA_QTN_21_ISLINECOMMENT_Diminuiu,DIFERENCA_QTN_21_ISLINECOMMENT_Manteve
0,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0.0,0.0,0.0,0,0,1,0,0,1
1,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0.0,0.0,0.0,0,0,1,0,0,1
2,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0.0,0.0,0.0,0,0,1,0,0,1
3,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0.0,0.0,0.0,0,0,1,0,0,1
4,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0.0,0.0,0.0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
273047,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,0.0,0.0,1.0,0,0,1,0,0,1
273048,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,-3.0,0.0,0.0,0,1,0,0,1,0
273049,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,0.0,0.0,0.0,0,0,1,0,0,1
273050,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,1.0,0.0,1.0,0,0,1,1,0,0


In [40]:
# Criar uma nova coluna 'COMMENTS_CHANGE' com valores padrão como None
df['DIFERENCA_QNT_18_SEGMENTOS_CHANGE'] = None
df['DIFERENCA_QNT_18_SEGMENTOS_Aumentou'] = None
df['DIFERENCA_QNT_18_SEGMENTOS_Diminuiu'] = None
df['DIFERENCA_QNT_18_SEGMENTOS_Manteve'] = None

# Atualizar a coluna 'DIFERENCA_QNT_18_SEGMENTOS_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QNT_18_SEGMENTOS'] > 0, 'DIFERENCA_QNT_18_SEGMENTOS_Aumentou'] = 1
df.loc[df['DIFERENCA_QNT_18_SEGMENTOS'] < 0, 'DIFERENCA_QNT_18_SEGMENTOS_Diminuiu'] = 1
df.loc[df['DIFERENCA_QNT_18_SEGMENTOS'] == 0, 'DIFERENCA_QNT_18_SEGMENTOS_Manteve'] = 1

# Atualizar a coluna 'DIFERENCA_QNT_18_SEGMENTOS_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QNT_18_SEGMENTOS'] > 0, 'DIFERENCA_QNT_18_SEGMENTOS_CHANGE'] = df['DIFERENCA_QNT_18_SEGMENTOS']
df.loc[df['DIFERENCA_QNT_18_SEGMENTOS'] < 0, 'DIFERENCA_QNT_18_SEGMENTOS_CHANGE'] = df['DIFERENCA_QNT_18_SEGMENTOS']
df.loc[df['DIFERENCA_QNT_18_SEGMENTOS'] == 0, 'DIFERENCA_QNT_18_SEGMENTOS_CHANGE'] = 0

df['DIFERENCA_QNT_18_SEGMENTOS_CHANGE'].fillna(0, inplace=True)
df['DIFERENCA_QNT_18_SEGMENTOS_Aumentou'].fillna(0, inplace=True)
df['DIFERENCA_QNT_18_SEGMENTOS_Diminuiu'].fillna(0, inplace=True)
df['DIFERENCA_QNT_18_SEGMENTOS_Manteve'].fillna(0, inplace=True)
df.drop(columns=['DIFERENCA_QNT_18_SEGMENTOS', 'DIFERENCA_QNT_18_SEGMENTOS_CHANGE'], inplace=True)
df

C:\Users\kleit\AppData\Local\Temp\ipykernel_11544\3125005421.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DIFERENCA_QNT_18_SEGMENTOS_CHANGE'] = None
C:\Users\kleit\AppData\Local\Temp\ipykernel_11544\3125005421.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DIFERENCA_QNT_18_SEGMENTOS_Aumentou'] = None
C:\Users\kleit\AppData\Local\Temp\ipykernel_11544\3125005421.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

,13_HASH_CLASSPATH,DIFERENCA_QTN_22_ISJAVADOCCOMMENT,DIFERENCA_QTN_23_ISORPHANCOMMENTA,DIFERENCA_QNT_20_ISBLOCKCOMMENT_Aumentou,DIFERENCA_QNT_20_ISBLOCKCOMMENT_Diminuiu,DIFERENCA_QNT_20_ISBLOCKCOMMENT_Manteve,DIFERENCA_QTN_21_ISLINECOMMENT_Aumentou,DIFERENCA_QTN_21_ISLINECOMMENT_Diminuiu,DIFERENCA_QTN_21_ISLINECOMMENT_Manteve,DIFERENCA_QNT_18_SEGMENTOS_Aumentou,DIFERENCA_QNT_18_SEGMENTOS_Diminuiu,DIFERENCA_QNT_18_SEGMENTOS_Manteve
0,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0.0,0.0,0,0,1,0,0,1,0,0,1
1,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0.0,0.0,0,0,1,0,0,1,0,0,1
2,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0.0,0.0,0,0,1,0,0,1,0,0,1
3,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0.0,0.0,0,0,1,0,0,1,0,0,1
4,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0.0,0.0,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
273047,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,0.0,1.0,0,0,1,0,0,1,0,0,1
273048,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,0.0,0.0,0,1,0,0,1,0,0,1,0
273049,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,0.0,0.0,0,0,1,0,0,1,0,0,1
273050,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,0.0,1.0,0,0,1,1,0,0,1,0,0


In [41]:
# Criar uma nova coluna 'COMMENTS_CHANGE' com valores padrão como None
df['DIFERENCA_QTN_22_ISJAVADOCCOMMENT_CHANGE'] = None
df['DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Aumentou'] = None
df['DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Diminuiu'] = None
df['DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Manteve'] = None

# Atualizar a coluna 'DIFERENCA_QTN_22_ISJAVADOCCOMMENT_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QTN_22_ISJAVADOCCOMMENT'] > 0, 'DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Aumentou'] = 1
df.loc[df['DIFERENCA_QTN_22_ISJAVADOCCOMMENT'] < 0, 'DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Diminuiu'] = 1
df.loc[df['DIFERENCA_QTN_22_ISJAVADOCCOMMENT'] == 0, 'DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Manteve'] = 1

# Atualizar a coluna 'DIFERENCA_QTN_22_ISJAVADOCCOMMENT_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QTN_22_ISJAVADOCCOMMENT'] > 0, 'DIFERENCA_QTN_22_ISJAVADOCCOMMENT_CHANGE'] = df['DIFERENCA_QTN_22_ISJAVADOCCOMMENT']
df.loc[df['DIFERENCA_QTN_22_ISJAVADOCCOMMENT'] < 0, 'DIFERENCA_QTN_22_ISJAVADOCCOMMENT_CHANGE'] = df['DIFERENCA_QTN_22_ISJAVADOCCOMMENT']
df.loc[df['DIFERENCA_QTN_22_ISJAVADOCCOMMENT'] == 0, 'DIFERENCA_QTN_22_ISJAVADOCCOMMENT_CHANGE'] = 0

df['DIFERENCA_QTN_22_ISJAVADOCCOMMENT_CHANGE'].fillna(0, inplace=True)
df['DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Aumentou'].fillna(0, inplace=True)
df['DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Diminuiu'].fillna(0, inplace=True)
df['DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Manteve'].fillna(0, inplace=True)
df.drop(columns=['DIFERENCA_QTN_22_ISJAVADOCCOMMENT', 'DIFERENCA_QTN_22_ISJAVADOCCOMMENT_CHANGE'], inplace=True)
df

C:\Users\kleit\AppData\Local\Temp\ipykernel_11544\911268844.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DIFERENCA_QTN_22_ISJAVADOCCOMMENT_CHANGE'] = None
C:\Users\kleit\AppData\Local\Temp\ipykernel_11544\911268844.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Aumentou'] = None
C:\Users\kleit\AppData\Local\Temp\ipykernel_11544\911268844.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

,13_HASH_CLASSPATH,DIFERENCA_QTN_23_ISORPHANCOMMENTA,DIFERENCA_QNT_20_ISBLOCKCOMMENT_Aumentou,DIFERENCA_QNT_20_ISBLOCKCOMMENT_Diminuiu,DIFERENCA_QNT_20_ISBLOCKCOMMENT_Manteve,DIFERENCA_QTN_21_ISLINECOMMENT_Aumentou,DIFERENCA_QTN_21_ISLINECOMMENT_Diminuiu,DIFERENCA_QTN_21_ISLINECOMMENT_Manteve,DIFERENCA_QNT_18_SEGMENTOS_Aumentou,DIFERENCA_QNT_18_SEGMENTOS_Diminuiu,DIFERENCA_QNT_18_SEGMENTOS_Manteve,DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Aumentou,DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Diminuiu,DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Manteve
0,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0.0,0,0,1,0,0,1,0,0,1,0,0,1
1,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0.0,0,0,1,0,0,1,0,0,1,0,0,1
2,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0.0,0,0,1,0,0,1,0,0,1,0,0,1
3,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0.0,0,0,1,0,0,1,0,0,1,0,0,1
4,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0.0,0,0,1,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273047,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,1.0,0,0,1,0,0,1,0,0,1,0,0,1
273048,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,0.0,0,1,0,0,1,0,0,1,0,0,0,1
273049,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,0.0,0,0,1,0,0,1,0,0,1,0,0,1
273050,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,1.0,0,0,1,1,0,0,1,0,0,0,0,1


In [42]:
# Criar uma nova coluna 'COMMENTS_CHANGE' com valores padrão como None
df['DIFERENCA_QTN_23_ISORPHANCOMMENTA_CHANGE'] = None
df['DIFERENCA_QTN_23_ISORPHANCOMMENTA_Aumentou'] = None
df['DIFERENCA_QTN_23_ISORPHANCOMMENTA_Diminuiu'] = None
df['DIFERENCA_QTN_23_ISORPHANCOMMENTA_Manteve'] = None

# Atualizar a coluna 'DIFERENCA_QTN_23_ISORPHANCOMMENTA_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QTN_23_ISORPHANCOMMENTA'] > 0, 'DIFERENCA_QTN_23_ISORPHANCOMMENTA_Aumentou'] = 1
df.loc[df['DIFERENCA_QTN_23_ISORPHANCOMMENTA'] < 0, 'DIFERENCA_QTN_23_ISORPHANCOMMENTA_Diminuiu'] = 1
df.loc[df['DIFERENCA_QTN_23_ISORPHANCOMMENTA'] == 0, 'DIFERENCA_QTN_23_ISORPHANCOMMENTA_Manteve'] = 1

# Atualizar a coluna 'DIFERENCA_QTN_23_ISORPHANCOMMENTA_CHANGE' com base na diferença calculada
df.loc[df['DIFERENCA_QTN_23_ISORPHANCOMMENTA'] > 0, 'DIFERENCA_QTN_23_ISORPHANCOMMENTA_CHANGE'] = df['DIFERENCA_QTN_23_ISORPHANCOMMENTA']
df.loc[df['DIFERENCA_QTN_23_ISORPHANCOMMENTA'] < 0, 'DIFERENCA_QTN_23_ISORPHANCOMMENTA_CHANGE'] = df['DIFERENCA_QTN_23_ISORPHANCOMMENTA']
df.loc[df['DIFERENCA_QTN_23_ISORPHANCOMMENTA'] == 0, 'DIFERENCA_QTN_23_ISORPHANCOMMENTA_CHANGE'] = 0

df['DIFERENCA_QTN_23_ISORPHANCOMMENTA_CHANGE'].fillna(0, inplace=True)
df['DIFERENCA_QTN_23_ISORPHANCOMMENTA_Aumentou'].fillna(0, inplace=True)
df['DIFERENCA_QTN_23_ISORPHANCOMMENTA_Diminuiu'].fillna(0, inplace=True)
df['DIFERENCA_QTN_23_ISORPHANCOMMENTA_Manteve'].fillna(0, inplace=True)
df.drop(columns=['DIFERENCA_QTN_23_ISORPHANCOMMENTA', 'DIFERENCA_QTN_23_ISORPHANCOMMENTA_CHANGE'], inplace=True)
df

C:\Users\kleit\AppData\Local\Temp\ipykernel_11544\144399164.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DIFERENCA_QTN_23_ISORPHANCOMMENTA_CHANGE'] = None
C:\Users\kleit\AppData\Local\Temp\ipykernel_11544\144399164.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DIFERENCA_QTN_23_ISORPHANCOMMENTA_Aumentou'] = None
C:\Users\kleit\AppData\Local\Temp\ipykernel_11544\144399164.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

,13_HASH_CLASSPATH,DIFERENCA_QNT_20_ISBLOCKCOMMENT_Aumentou,DIFERENCA_QNT_20_ISBLOCKCOMMENT_Diminuiu,DIFERENCA_QNT_20_ISBLOCKCOMMENT_Manteve,DIFERENCA_QTN_21_ISLINECOMMENT_Aumentou,DIFERENCA_QTN_21_ISLINECOMMENT_Diminuiu,DIFERENCA_QTN_21_ISLINECOMMENT_Manteve,DIFERENCA_QNT_18_SEGMENTOS_Aumentou,DIFERENCA_QNT_18_SEGMENTOS_Diminuiu,DIFERENCA_QNT_18_SEGMENTOS_Manteve,DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Aumentou,DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Diminuiu,DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Manteve,DIFERENCA_QTN_23_ISORPHANCOMMENTA_Aumentou,DIFERENCA_QTN_23_ISORPHANCOMMENTA_Diminuiu,DIFERENCA_QTN_23_ISORPHANCOMMENTA_Manteve
0,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1
1,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1
2,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1
3,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1
4,000b10c37c94b8f8b3ba1abc35b5d586181be378/commo...,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273047,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0
273048,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1
273049,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1
273050,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0


# refactorings


In [43]:
DF_REFACTORING.rename(columns={'17_HASH_OLDPATHCLASS': '13_HASH_CLASSPATH'}, inplace=True)
DF_REFACTORING

,11_HASH,12_PARENTHASH,13_REFACTORINGTYPE,14_OCORRIDO,15_OLDPATHCLASS,16_NEWPATHCLASS,13_HASH_CLASSPATH
0,d2f91bf323e90cce24e679ce07ed23573bfd245f,5e42336048e2d79c37931bd7eef0209c532707d6,SPLIT_OPERATION,Split Method Split Method\tpublic kotlinWithDe...,value/src/it/functional/src/test/java/com/goog...,value/src/it/functional/src/test/java/com/goog...,d2f91bf323e90cce24e679ce07ed23573bfd245f/value...
1,d2f91bf323e90cce24e679ce07ed23573bfd245f,5e42336048e2d79c37931bd7eef0209c532707d6,SPLIT_OPERATION,Split Method Split Method\tpublic kotlinWithDe...,value/src/it/functional/src/test/java/com/goog...,value/src/it/functional/src/test/java/com/goog...,d2f91bf323e90cce24e679ce07ed23573bfd245f/value...
2,d2f91bf323e90cce24e679ce07ed23573bfd245f,5e42336048e2d79c37931bd7eef0209c532707d6,CHANGE_VARIABLE_TYPE,Change Variable Type Change Variable Type\tx :...,value/src/it/functional/src/test/java/com/goog...,value/src/it/functional/src/test/java/com/goog...,d2f91bf323e90cce24e679ce07ed23573bfd245f/value...
3,d2f91bf323e90cce24e679ce07ed23573bfd245f,5e42336048e2d79c37931bd7eef0209c532707d6,RENAME_VARIABLE,Rename Variable Rename Variable\tx : KotlinDat...,value/src/it/functional/src/test/java/com/goog...,value/src/it/functional/src/test/java/com/goog...,d2f91bf323e90cce24e679ce07ed23573bfd245f/value...
4,d2f91bf323e90cce24e679ce07ed23573bfd245f,5e42336048e2d79c37931bd7eef0209c532707d6,CHANGE_VARIABLE_TYPE,Change Variable Type Change Variable Type\trel...,value/src/main/java/com/google/auto/value/proc...,value/src/main/java/com/google/auto/value/proc...,d2f91bf323e90cce24e679ce07ed23573bfd245f/value...
...,...,...,...,...,...,...,...
5461,994ba302617596b4f8efc2ec1305a092609f01f3,07d6fdda4917e9fd734cd680b9aeabe3f3ab9e64,EXTRACT_VARIABLE,Extract Variable Extract Variable\ttypeQualifi...,factory/src/main/java/com/google/auto/factory/...,factory/src/main/java/com/google/auto/factory/...,994ba302617596b4f8efc2ec1305a092609f01f3/facto...
5462,994ba302617596b4f8efc2ec1305a092609f01f3,07d6fdda4917e9fd734cd680b9aeabe3f3ab9e64,RENAME_ATTRIBUTE,Rename Attribute Rename Attribute\tparameterTo...,factory/src/main/java/com/google/auto/factory/...,factory/src/main/java/com/google/auto/factory/...,994ba302617596b4f8efc2ec1305a092609f01f3/facto...
5463,994ba302617596b4f8efc2ec1305a092609f01f3,07d6fdda4917e9fd734cd680b9aeabe3f3ab9e64,ADD_ATTRIBUTE_MODIFIER,Add Attribute Modifier Add Attribute Modifier\...,factory/src/main/java/com/google/auto/factory/...,factory/src/main/java/com/google/auto/factory/...,994ba302617596b4f8efc2ec1305a092609f01f3/facto...
5464,30cea621cb6610acca6b39cd5f0d6805e9293546,4f9e65319dfb3ace864bb244371a86fe8b643dce,RENAME_PARAMETER,Rename Parameter Rename Parameter\ta : int to ...,factory/src/test/resources/expected/FactoryImp...,factory/src/test/resources/expected/FactoryImp...,30cea621cb6610acca6b39cd5f0d6805e9293546/facto...


In [44]:
pivoted = DF_REFACTORING.pivot_table(index='13_HASH_CLASSPATH', columns='13_REFACTORINGTYPE', aggfunc='size', fill_value=0)
pivoted

13_REFACTORINGTYPE,ADD_ATTRIBUTE_ANNOTATION,ADD_ATTRIBUTE_MODIFIER,ADD_CLASS_ANNOTATION,ADD_CLASS_MODIFIER,ADD_METHOD_ANNOTATION,ADD_METHOD_MODIFIER,ADD_PARAMETER,ADD_PARAMETER_ANNOTATION,ADD_PARAMETER_MODIFIER,ADD_THROWN_EXCEPTION_TYPE,...,REORDER_PARAMETER,REPLACE_ANONYMOUS_WITH_LAMBDA,REPLACE_ATTRIBUTE_WITH_VARIABLE,REPLACE_LOOP_WITH_PIPELINE,REPLACE_VARIABLE_WITH_ATTRIBUTE,SPLIT_ATTRIBUTE,SPLIT_CONDITIONAL,SPLIT_OPERATION,SPLIT_PARAMETER,SPLIT_VARIABLE
13_HASH_CLASSPATH,,,,,,,,,,,,,,,,,,,,,
0076afb3825bd4d3b7ab77310855c2cc4a05813e/value/src/main/java/com/google/auto/value/processor/AutoValueOrOneOfProcessor.java,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0076afb3825bd4d3b7ab77310855c2cc4a05813e/value/src/main/java/com/google/auto/value/processor/AutoValueOrOneOfTemplateVars.java,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0076afb3825bd4d3b7ab77310855c2cc4a05813e/value/src/main/java/com/google/auto/value/processor/AutoValueProcessor.java,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0076afb3825bd4d3b7ab77310855c2cc4a05813e/value/src/main/java/com/google/auto/value/processor/AutoValueTemplateVars.java,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0076afb3825bd4d3b7ab77310855c2cc4a05813e/value/src/main/java/com/google/auto/value/processor/BuilderSpec.java,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ff523aeab1ec8f5f3ad821e2d75bd3d336dd8096/factory/src/main/java/com/google/auto/factory/processor/FactoryDescriptor.java,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ff523aeab1ec8f5f3ad821e2d75bd3d336dd8096/factory/src/main/java/com/google/auto/factory/processor/FactoryWriter.java,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ff523aeab1ec8f5f3ad821e2d75bd3d336dd8096/factory/src/main/java/com/google/auto/factory/processor/Key.java,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
tabela_completa = pd.merge(pivoted, df, on='13_HASH_CLASSPATH', how='inner')
tabela_completa.to_csv('tabela_completa_' + FILE_NAME_COMMENTS, index=False)
tabela_completa

,13_HASH_CLASSPATH,ADD_ATTRIBUTE_ANNOTATION,ADD_ATTRIBUTE_MODIFIER,ADD_CLASS_ANNOTATION,ADD_CLASS_MODIFIER,ADD_METHOD_ANNOTATION,ADD_METHOD_MODIFIER,ADD_PARAMETER,ADD_PARAMETER_ANNOTATION,ADD_PARAMETER_MODIFIER,...,DIFERENCA_QTN_21_ISLINECOMMENT_Manteve,DIFERENCA_QNT_18_SEGMENTOS_Aumentou,DIFERENCA_QNT_18_SEGMENTOS_Diminuiu,DIFERENCA_QNT_18_SEGMENTOS_Manteve,DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Aumentou,DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Diminuiu,DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Manteve,DIFERENCA_QTN_23_ISORPHANCOMMENTA_Aumentou,DIFERENCA_QTN_23_ISORPHANCOMMENTA_Diminuiu,DIFERENCA_QTN_23_ISORPHANCOMMENTA_Manteve
0,0076afb3825bd4d3b7ab77310855c2cc4a05813e/value...,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,1
1,0076afb3825bd4d3b7ab77310855c2cc4a05813e/value...,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,1,0,0,0,1
2,0076afb3825bd4d3b7ab77310855c2cc4a05813e/value...,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,1
3,0076afb3825bd4d3b7ab77310855c2cc4a05813e/value...,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,1,0,0,1
4,0076afb3825bd4d3b7ab77310855c2cc4a05813e/value...,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1014,ff523aeab1ec8f5f3ad821e2d75bd3d336dd8096/facto...,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,1
1015,ff523aeab1ec8f5f3ad821e2d75bd3d336dd8096/facto...,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
1016,ff523aeab1ec8f5f3ad821e2d75bd3d336dd8096/facto...,0,0,0,1,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,1
1017,ffc75cd6daf284bac1e9bc03a01391283a506dfa/test-...,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,1,0,0


In [46]:
tabela_completa.drop(columns=['13_HASH_CLASSPATH'], inplace=True)
tabela_completa

,ADD_ATTRIBUTE_ANNOTATION,ADD_ATTRIBUTE_MODIFIER,ADD_CLASS_ANNOTATION,ADD_CLASS_MODIFIER,ADD_METHOD_ANNOTATION,ADD_METHOD_MODIFIER,ADD_PARAMETER,ADD_PARAMETER_ANNOTATION,ADD_PARAMETER_MODIFIER,ADD_THROWN_EXCEPTION_TYPE,...,DIFERENCA_QTN_21_ISLINECOMMENT_Manteve,DIFERENCA_QNT_18_SEGMENTOS_Aumentou,DIFERENCA_QNT_18_SEGMENTOS_Diminuiu,DIFERENCA_QNT_18_SEGMENTOS_Manteve,DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Aumentou,DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Diminuiu,DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Manteve,DIFERENCA_QTN_23_ISORPHANCOMMENTA_Aumentou,DIFERENCA_QTN_23_ISORPHANCOMMENTA_Diminuiu,DIFERENCA_QTN_23_ISORPHANCOMMENTA_Manteve
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,1,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,1,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1014,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,1
1015,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
1016,0,0,0,1,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,1
1017,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,1,0,0


In [47]:
def transforma_valor(valor):
    if valor > 0:
        return 1
    else:
        return valor

# Aplica a função em cada elemento do DataFrame
tabela_completa = tabela_completa.applymap(transforma_valor)

C:\Users\kleit\AppData\Local\Temp\ipykernel_11544\1884997101.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  tabela_completa = tabela_completa.applymap(transforma_valor)


In [48]:
tabela_completa

,ADD_ATTRIBUTE_ANNOTATION,ADD_ATTRIBUTE_MODIFIER,ADD_CLASS_ANNOTATION,ADD_CLASS_MODIFIER,ADD_METHOD_ANNOTATION,ADD_METHOD_MODIFIER,ADD_PARAMETER,ADD_PARAMETER_ANNOTATION,ADD_PARAMETER_MODIFIER,ADD_THROWN_EXCEPTION_TYPE,...,DIFERENCA_QTN_21_ISLINECOMMENT_Manteve,DIFERENCA_QNT_18_SEGMENTOS_Aumentou,DIFERENCA_QNT_18_SEGMENTOS_Diminuiu,DIFERENCA_QNT_18_SEGMENTOS_Manteve,DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Aumentou,DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Diminuiu,DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Manteve,DIFERENCA_QTN_23_ISORPHANCOMMENTA_Aumentou,DIFERENCA_QTN_23_ISORPHANCOMMENTA_Diminuiu,DIFERENCA_QTN_23_ISORPHANCOMMENTA_Manteve
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,1,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,1,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1014,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,1
1015,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
1016,0,0,0,1,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,1
1017,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,1,0,0


In [49]:
itemsets = apriori(tabela_completa, min_support=0.1, use_colnames=True, max_len=2)


c:\Python312\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


# Regras

In [53]:

rules = association_rules(itemsets, metric="confidence", min_threshold=0.01)
rules.to_csv('regras projeto auto - com java doc.csv', index=False)

In [51]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(ADD_PARAMETER),(DIFERENCA_QNT_20_ISBLOCKCOMMENT_Manteve),0.186457,0.972522,0.174681,0.936842,0.963312,-0.006653,0.435067,-0.044721
1,(DIFERENCA_QNT_20_ISBLOCKCOMMENT_Manteve),(ADD_PARAMETER),0.972522,0.186457,0.174681,0.179617,0.963312,-0.006653,0.991662,-0.580895
2,(ADD_PARAMETER),(DIFERENCA_QTN_21_ISLINECOMMENT_Manteve),0.186457,0.737978,0.141315,0.757895,1.026988,0.003714,1.082263,0.032301
3,(DIFERENCA_QTN_21_ISLINECOMMENT_Manteve),(ADD_PARAMETER),0.737978,0.186457,0.141315,0.191489,1.026988,0.003714,1.006224,0.100291
4,(ADD_PARAMETER),(DIFERENCA_QNT_18_SEGMENTOS_Manteve),0.186457,0.630029,0.119725,0.642105,1.019167,0.002252,1.033741,0.023117
...,...,...,...,...,...,...,...,...,...,...
81,(DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Manteve),(DIFERENCA_QNT_18_SEGMENTOS_Manteve),0.847890,0.630029,0.630029,0.743056,1.179398,0.095834,1.439885,1.000000
82,(DIFERENCA_QTN_23_ISORPHANCOMMENTA_Manteve),(DIFERENCA_QNT_18_SEGMENTOS_Manteve),0.806673,0.630029,0.627085,0.777372,1.233867,0.118858,1.661835,0.980410
83,(DIFERENCA_QNT_18_SEGMENTOS_Manteve),(DIFERENCA_QTN_23_ISORPHANCOMMENTA_Manteve),0.630029,0.806673,0.627085,0.995327,1.233867,0.118858,41.371933,0.512310
84,(DIFERENCA_QTN_23_ISORPHANCOMMENTA_Manteve),(DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Manteve),0.806673,0.847890,0.727184,0.901460,1.063180,0.043213,1.543634,0.307384


In [52]:
tabela_completa

,ADD_ATTRIBUTE_ANNOTATION,ADD_ATTRIBUTE_MODIFIER,ADD_CLASS_ANNOTATION,ADD_CLASS_MODIFIER,ADD_METHOD_ANNOTATION,ADD_METHOD_MODIFIER,ADD_PARAMETER,ADD_PARAMETER_ANNOTATION,ADD_PARAMETER_MODIFIER,ADD_THROWN_EXCEPTION_TYPE,...,DIFERENCA_QTN_21_ISLINECOMMENT_Manteve,DIFERENCA_QNT_18_SEGMENTOS_Aumentou,DIFERENCA_QNT_18_SEGMENTOS_Diminuiu,DIFERENCA_QNT_18_SEGMENTOS_Manteve,DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Aumentou,DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Diminuiu,DIFERENCA_QTN_22_ISJAVADOCCOMMENT_Manteve,DIFERENCA_QTN_23_ISORPHANCOMMENTA_Aumentou,DIFERENCA_QTN_23_ISORPHANCOMMENTA_Diminuiu,DIFERENCA_QTN_23_ISORPHANCOMMENTA_Manteve
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,1,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,1,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1014,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,1
1015,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
1016,0,0,0,1,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,1
1017,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,1,0,0
